# Recommender Systems

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!



In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=20, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   3.0|    28| 2.3009248|
|      3|   1.0|     1|  1.067014|
|      4|   2.0|     1| 1.9812715|
|      0|   1.0|    20|0.80433196|
|      5|   1.0|     5| 1.2989111|
|      0|   1.0|    15| 0.6072106|
|      1|   4.0|    15|  1.311063|
|      5|   2.0|    15| 1.2469846|
|      2|   1.0|    17| 1.1405857|
|      2|   3.0|     9| 2.0552733|
|      5|   1.0|     9|0.21402667|
|      2|   4.0|     8|  3.450018|
|      4|   2.0|     8| 1.4765621|
|      5|   1.0|     8| 1.2525373|
|      1|   1.0|     7| 1.3079714|
|      3|   1.0|     7| 1.5636817|
|      4|   1.0|     7| 1.6066747|
|      2|   4.0|    10|   2.59993|
|      4|   1.0|    29| 1.3252573|
|      1|   1.0|    14|0.72039497|
+-------+------+------+----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.027235376945848


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user = test.filter(test['userId']==11)

In [14]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|     13|   4.0|    11|
|     45|   1.0|    11|
|     51|   3.0|    11|
|     67|   1.0|    11|
|     72|   3.0|    11|
|     82|   1.0|    11|
|     90|   4.0|    11|
|     97|   3.0|    11|
+-------+------+------+



In [15]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     51|   3.0|    11|  2.540942|
|     13|   4.0|    11| 2.5298102|
|     90|   4.0|    11| 2.1272955|
|     72|   3.0|    11| 2.0292518|
|     45|   1.0|    11| 1.4258696|
|     82|   1.0|    11| 1.3385773|
|     97|   3.0|    11| 1.1395739|
|     67|   1.0|    11| 0.7819144|
+-------+------+------+----------+



# Great Job!